In [ ]:
from scipy.signal import find_peaks
from scipy.stats import pearsonr
import numpy as np

# Assuming y_test and test_predictions are already defined
nse_values = []
rsr_values = []
evol_values = []
tpe_values = []
kge_value = []

for step in range(n_steps_out):
    observed = y_test[:, step]
    predicted = test_predictions[:, step]

    # Nash-Sutcliffe Efficiency (NSE)
    numerator = np.sum((observed - predicted) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)
    nse = 1 - (numerator / denominator)
    nse_values.append(nse)

    # R-Squared Reliability (RSR)
    rsr = np.sqrt(numerator / denominator)
    rsr_values.append(rsr)

    # Evolution Criterion (Evol)
    evol = ((np.sum(predicted) - np.sum(observed)) / np.sum(observed)) * 100
    evol_values.append(evol)

    # Modified Total Percent Error (TPE) using top 20% peaks
    peaks, _ = find_peaks(observed)
    peak_values = observed[peaks]
    num_top_peaks = max(1, int(0.2 * len(peak_values)))
    top_peak_indices = peaks[np.argsort(peak_values)[-num_top_peaks:]]
    observed_top = observed[top_peak_indices]
    predicted_top = predicted[top_peak_indices]
    numerator_tpe = np.sum(np.abs(observed_top - predicted_top))
    denominator_tpe = np.sum(observed_top)
    tpe = numerator_tpe / denominator_tpe
    tpe_values.append(tpe)

    # Kling-Gupta Efficiency (KGE)
    r, _ = pearsonr(predicted, observed)
    mean_simulated = np.mean(predicted)
    mean_observed = np.mean(observed)
    std_simulated = np.std(predicted)
    std_observed = np.std(observed)
    kge = 1 - np.sqrt(
        (r - 1)**2 +
        ((std_simulated / std_observed) - 1)**2 +
        ((mean_simulated / mean_observed) - 1)**2
    )
    kge_value.append(kge)
# Print metrics for all timesteps
print("NSE Values:", np.round(nse_values, 4))
print("RSR Values:", np.round(rsr_values, 4))
print("Evol Values:", np.round(evol_values, 4))
print("PE Values:", np.round(tpe_values, 4))
print("KGE Values:", np.round(kge_value, 4))
